# PAPER PEEK📜👀

The purpose of the project is to build an NLP model to make reading medical abstracts easier.

The paper we're replicating (the source of the dataset that we'll be using) is available here: https://arxiv.org/abs/1710.06071

Reading through the paper above, we see that the model architecture that they use to achieve their best results is available here: https://arxiv.org/abs/1612.05251

## Confirm access to a GPU 

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce MX330 (UUID: GPU-77eebd61-350b-2f9c-b351-0acab8f44c51)


## Get data

Since we'll be replicating the paper above (PubMed 200k RCT), let's download the dataset they used.

We can do so from the authors GitHub: https://github.com/Franck-Dernoncourt/pubmed-rct 

In [ ]:
# I have used the zip file but you can run below command which will clone the github repository and import the dataset.
# !git clone https://github.com/Franck-Dernoncourt/pubmed-rct
# !ls pubmed-rct

#OUTPUT:
# Cloning into 'pubmed-rct'...
# remote: Enumerating objects: 33, done.
# remote: Counting objects: 100% (8/8), done.
# remote: Compressing objects: 100% (3/3), done.
# remote: Total 33 (delta 5), reused 5 (delta 5), pack-reused 25
# Unpacking objects: 100% (33/33), 177.08 MiB | 14.49 MiB/s, done.
# PubMed_200k_RCT
# PubMed_200k_RCT_numbers_replaced_with_at_sign
# PubMed_20k_RCT
# PubMed_20k_RCT_numbers_replaced_with_at_sign
# README.md

In [ ]:
# Check what files are in the PubMed_20K dataset
# !ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

#OUTPUT:
# dev.txt  test.txt  train.txt

In [8]:
# Start our experiments using the 20k dataset with numbers replaced by "@" sign
data_dir = "pubmed-rct-master/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [9]:
# Check all of the filenames in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

['pubmed-rct-master/PubMed_20k_RCT_numbers_replaced_with_at_sign/dev.txt',
 'pubmed-rct-master/PubMed_20k_RCT_numbers_replaced_with_at_sign/test.txt',
 'pubmed-rct-master/PubMed_20k_RCT_numbers_replaced_with_at_sign/train.txt']